In [1]:
import glob
import json
from tqdm.notebook import tqdm

In [61]:
# gather all json files
files = glob.glob('/Users/codyfalkosky/Desktop/2_cleaned_data/*')

In [3]:
from transformers import AutoTokenizer

token = ''
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
class ToMessage:
    def __init__(self, data, heading_message_format='by heading'):
        self.data = data
        self.heading_message_format = heading_message_format

        # init
        self.title = None
        self.headings = []
        self.title_message = []
        self.heading_messages = []
        self.blog = ''

        # extract
        self._get_title_and_headings()

        # compile messages
        self._generate_messages()

    def _get_title_and_headings(self):
        for example in self.data:
            if {'title', 'headings'} <= example.keys():
                self.title = example['title']
            if {'heading', 'content'} <= example.keys():
                self.headings.append(example['heading'])
                
    def _generate_title_message(self):
        m = [
            {"role": "system", "content": "You are a helpful AI assistant for writing blog headings"},
            {"role": "user", "content": f'''Generate headings for a blog titled "{self.title}"'''},
            {"role": "assistant", "content": f"{self.headings}"},
        ]

        self.title_message.append({'messages':m, 'label':'title'})

    def _generate_heading_message(self, example):
        m = [
            {"role": "system", "content": "You are a helpful AI assistant for writing blog content"},
            {"role": "user", "content": f'''Given a blog with title: "{self.title}", and headings: "{self.headings}", generate the content that belongs to the specific heading: "{example['heading']}."'''},
            {"role": "assistant", "content": example['content']},
        ]

        self.heading_messages.append({'messages':m, 'label':'heading'})

    def _generate_blog_message(self):
        m = [
            {"role": "system", "content": "You are a helpful AI assistant for writing entire blogs"},
            {"role": "user", "content": f'''Given a blog with title: "{self.title}", and headings: "{self.headings}", generate and write all content for the full blog."'''},
            {"role": "assistant", "content": self.blog},
        ]

        self.heading_messages.append({'messages':m, 'label':'blog'})

    def _generate_messages(self):
        self._generate_title_message()

        if self.heading_message_format == 'by heading':
            for example in self.data:
                if {'heading', 'content'} <= example.keys():
                    self._generate_heading_message(example)

        if self.heading_message_format == 'full blog':
            # self.blog += self.title + '\n\n' # already in most headings
            
            for example in self.data:
                if {'heading', 'content'} <= example.keys():
                    self.blog += example['heading'] + '\n'
                    self.blog += example['content'] + '\n\n'

            self.blog = self.blog.strip()
            self._generate_blog_message()
                    

## Generate Messages

In [41]:
from transformers import AutoTokenizer

token = ''
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [42]:
## Compile all messages

title_messages = []
blog_messages  = []

In [43]:
for path in tqdm(files):
    
    with open(path, 'r') as file:
        DATA = json.load(file)

    for data in DATA:
        # format
        to_message = ToMessage(data, heading_message_format='full blog')

        # add to main list
        title_messages.extend(to_message.title_message)

        # check message length
        messages = to_message.heading_messages
        tokens = tokenizer.apply_chat_template(messages[0]['messages'])
        if len(tokens) < 8192:
            blog_messages.extend(to_message.heading_messages)

  0%|          | 0/24 [00:00<?, ?it/s]

In [24]:
with open('/Users/codyfalkosky/Desktop/3_message_format/full_blog_matching_only_message_labeled_8192/title_messages_labeled.json', 'w') as file:
    json.dump(title_messages, file)

In [25]:
with open('/Users/codyfalkosky/Desktop/3_message_format/full_blog_matching_only_message_labeled_8192/blog_messages_labeled.json', 'w') as file:
    json.dump(blog_messages, file)

## Inspect

In [38]:
path = '/Users/codyfalkosky/Desktop/3_message_format/full_blog_matching_only/blog_messages.json'

with open(path, 'r') as file:
    for_inspect = json.load(file)

In [45]:
print(for_inspect[0][0], '\n')
print(for_inspect[0][1], '\n')
print(for_inspect[0][2]['content'])

{'role': 'system', 'content': 'You are a helpful AI assistant for writing entire blogs'} 

{'role': 'user', 'content': 'Given a blog with title: "18 Days Beautiful Remote Part of Mexico", and headings: "[\'<h2>18 Days in a Super Beautiful, Remote Part of Mexico</h2>\', \'<h3>The Routine</h3>\', \'<h3>The Reality</h3>\', \'<h3>The Solution</h3>\']", generate and write all content for the full blog."'} 

<h2>18 Days in a Super Beautiful, Remote Part of Mexico</h2>
Last month, I spent 18 days in a secluded, remote part of Mexico. A friend and I were covering another friend's housesitting assignment while he and his wife were away. The house we took care of was quite isolated. On one side was an endless, empty white sand beach and gorgeous water for as far as you could see. On the other side was an immense span of jungle stretching all the way to Alaska, or so it seemed. Nothing else at all for the most part. A potholed dirt road was the only way in and out. The closest town was 25 kms awa